In [42]:
from numpy.fft import fft
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [43]:
N_data = 2500  # Number of training data in final augmented dataset
L = 1024      # Length of IQ sequence in each sample
D = 100 # Decimation factor for long-term data
SNRdBmin = 10
SNRdBmax = 20

max_pw  = 1000 # Pulse Width (us)
max_pri = 1000 # Pulse Repetition Interval (us)
min_sig = 0.01
max_sig = 1    # Magnitude of the radar signal
SNRdBmin = 0
SNRdBmax = 20
fs = 20e6
LFM_maxBW = 10e6
LFM_minBW = 1e6
max_agility_BW = 5e6
min_agility_BW = 1e6
seq_len = 1000

checkpoint_filepath = 'temp/checkpoint/freq_agility_1.h5'
log_filepath = 'temp/tb_logs'

N_train_iter = 100

In [44]:
def gen_data():
    data = np.zeros((N_data, L, 8))

    # 9 columns: 
    #   0: modulation (0=unmodulated, 1=linear frequency modulation) 
    #   1: frequency agility (0=no agility, 1=jitter)
    #   2: pulse width agility (0=no agility, 1=jitter)
    #   3: pulse repetition interval agility (0=no agility,1=jitter)
    #   4: minimum pulse width
    #   5: maximum pulse width (same as minimum if no agility)
    #   6: minimum pulse repetition interval
    #   7: maximum pulse repetition interval (same as minimum if no agility)
    #   8: SNR in dB
    labels = np.zeros((N_data, 9)) 

    for i in range(N_data):
        # Select Radar emitter parameters
        pri_min = random.uniform(0, np.log10(max_pri)) 
        pri_min = 10**pri_min
        pw_max = random.uniform(0, np.log10(min(pri_min, max_pw))) # PW needs to be less than pri, or the radar will be CW
        pw_max = 10**pw_max
        sig_mag = random.uniform(np.log10(min_sig), np.log10(max_sig))
        sig_mag = 10**sig_mag
        phase_off = random.uniform(0, 2*np.pi)

        pulse_phase = random.uniform(0, 1) # Phase between 0 and 1 as percent of cycle elapsed at start of waveform
        mod_type = random.randint(0,1) # 0 is unmodulated, 1 is LFM
        freq_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        pw_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        pri_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        freq_agility_BW = random.uniform(min_agility_BW, max_agility_BW)
        LFM_BW = random.uniform(np.log10(LFM_maxBW), np.log10(LFM_minBW))
        LFM_BW = 10**LFM_BW

        freq_seq = np.zeros(seq_len)
        freq_off_seq = np.zeros(seq_len)
        if (freq_agile == 1):
            for j in range(seq_len):
                freq_seq[j] = random.uniform(-LFM_BW/2, LFM_BW/2) # deviation of frequency from center frequency
                freq_off_seq[j] = random.uniform(0, 2*np.pi) # phase offset that varies per pulse

        pw_seq = np.zeros(seq_len)
        if (pw_agile == 1):
            pw_min = random.uniform(np.log10(1), np.log10(pw_max))
            pw_min = 10**pw_min
            for j in range(seq_len):
                pw_seq[j] = random.uniform(pw_min, pw_max) 
        else:
            pw_min = pw_max

        pri_seq = np.zeros(seq_len)
        if (pri_agile == 1):
            pri_max = random.uniform(np.log10(pri_min), np.log10(max_pri))
            pri_max = 10**pri_max
            for j in range(seq_len):
                pri_seq[j] = random.uniform(pri_min, pri_max)
        else:
            pri_max = pri_min


        t = np.linspace(0.0, (L*D-1)/fs, L*D)
        dt = t[1]-t[0]

        x_mag = np.zeros(D*L)
        x_phase = np.zeros(D*L)
        # No modulation or agility
        if (mod_type == 0) & (pw_agile == 0) & (pri_agile == 0) & (freq_agile == 0):
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_phase = phase_off
        # basic LFM
        elif (mod_type ==1) & (pw_agile == 0) & (pri_agile == 0) & (freq_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_min*1e-6)-x_time)
        # unmod freq_agile
        elif (mod_type ==0) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int) # Turn the floats in the array into integers so that they can be used for indexing. Lord only knows why integer division returns a float
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # LFM freq_agile
        elif (mod_type ==1) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_min*1e-6)-x_time)+2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # unmod pw_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_phase = phase_off
        # LFM pw_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_seq[k % seq_len]*1e-6)-x_time)
        # unmod pw_agile freq_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # LFM pw_agile freq_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_seq[k % seq_len]*1e-6)-x_time) + 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # pri_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = phase_off
        # LFM pri_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)
        # freq_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # LFM freq_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)+2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # pw_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = phase_off
        # LFM pw_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_seq[k % seq_len]*1e-6)-x_time_p)
        # freq_agile pw_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # LFM freq_agile pw_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)+2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        

        x = x_mag*np.exp(1j*x_phase)
        x = sig_mag*x

        # Add AWGN noise with random power
        SNRdB = random.uniform(SNRdBmin, SNRdBmax)
        SNR = 10**(SNRdB/10)
        sig_pwr = np.mean(np.abs(x)**2)
        sigma2 = sig_pwr/SNR
        noise = np.random.randn(np.size(x))*np.sqrt(sigma2/2) + 1j*np.random.randn(np.size(x))*np.sqrt(sigma2/2)
        x_long = x + noise

        x_short = x_long[np.arange(0, int(np.size(x_long)/D), 1)]
        x_long = x_long[np.arange(0, np.size(x_long), D)]

        xf_short = fft(x_short)
        xf_long  = fft(x_long)


        x_ms = abs(x_short)
        x_ps = np.angle(x_short)
        x_ml = abs(x_long)
        x_pl = np.angle(x_long)
        xf_ms = abs(xf_short)
        xf_ps = np.angle(xf_short)
        xf_ml = abs(xf_long)
        xf_pl = np.angle(xf_long)

        data[i,:,0] = x_ms
        data[i,:,1] = x_ps
        data[i,:,2] = x_ml
        data[i,:,3] = x_pl
        data[i,:,4] = xf_ms
        data[i,:,5] = xf_ps
        data[i,:,6] = xf_ml
        data[i,:,7] = xf_pl

        labels[i,0] = mod_type
        labels[i,1] = freq_agile
        labels[i,2] = pw_agile
        labels[i,3] = pri_agile
        labels[i,4] = pw_min
        labels[i,5] = pw_max
        labels[i,6] = pri_min
        labels[i,7] = pri_max
        labels[i,8] = SNRdB

    return data, labels

In [45]:
def build_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size, activation='relu'))
    return model

In [46]:
# NOT DONE
def build_func_model():
    X1=Conv1D(10, kernel_size=5, padding='same', input_shape=(1024,8), activation='relu')
    X2=MaxPooling1D(pool_size=2, strides=2)(X1)
    X3=Conv1D(20, kernel_size=5, padding='same', activation='relu')(X2)
    X4=MaxPooling1D(pool_size=2, strides=2)(X3)
    X5=Conv1D(30, kernel_size=5, padding='same', activation='relu')(X4)
    X6=MaxPooling1D(pool_size=2, strides=2)(X5)

In [47]:
model = build_model(input_shape=(1024,8),output_size=2)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_36 (Conv1D)          (None, 1024, 10)          410       
                                                                 
 max_pooling1d_36 (MaxPooli  (None, 512, 10)           0         
 ng1D)                                                           
                                                                 
 conv1d_37 (Conv1D)          (None, 512, 20)           1020      
                                                                 
 max_pooling1d_37 (MaxPooli  (None, 256, 20)           0         
 ng1D)                                                           
                                                                 
 conv1d_38 (Conv1D)          (None, 256, 30)           3030      
                                                                 
 max_pooling1d_38 (MaxPooli  (None, 128, 30)          

In [48]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
# Setup the callbacks
checkpt = ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=0)
tb = TensorBoard(log_dir=log_filepath)
e_stop = EarlyStopping(patience=5)

In [49]:
#data, labels = gen_data()

In [50]:
# Grab pw agility data
#y=labels[:,2]
#y

In [51]:
for i in range(N_train_iter):
    x, labels = gen_data()
    y = labels[:,1] # freq agility data
    
    # Train the model
    hist = model.fit(x, y, epochs=100, validation_split=0.2, verbose=2, batch_size=100,
                    callbacks=[checkpt, tb, e_stop])
    

Epoch 1/100
20/20 - 3s - loss: 0.7175 - accuracy: 0.4920 - val_loss: 0.6931 - val_accuracy: 0.5220 - 3s/epoch - 159ms/step
Epoch 2/100
20/20 - 0s - loss: 0.6931 - accuracy: 0.4930 - val_loss: 0.6931 - val_accuracy: 0.5220 - 398ms/epoch - 20ms/step
Epoch 3/100
20/20 - 0s - loss: 0.6931 - accuracy: 0.4930 - val_loss: 0.6931 - val_accuracy: 0.5220 - 390ms/epoch - 20ms/step
Epoch 4/100
20/20 - 0s - loss: 0.6931 - accuracy: 0.4930 - val_loss: 0.6931 - val_accuracy: 0.5220 - 424ms/epoch - 21ms/step
Epoch 5/100
20/20 - 0s - loss: 0.6931 - accuracy: 0.4930 - val_loss: 0.6931 - val_accuracy: 0.5220 - 425ms/epoch - 21ms/step
Epoch 6/100
20/20 - 0s - loss: 0.6931 - accuracy: 0.4930 - val_loss: 0.6931 - val_accuracy: 0.5220 - 424ms/epoch - 21ms/step
Epoch 1/100
20/20 - 1s - loss: 0.6931 - accuracy: 0.5130 - val_loss: 0.6931 - val_accuracy: 0.5140 - 702ms/epoch - 35ms/step
Epoch 2/100
20/20 - 0s - loss: 0.6931 - accuracy: 0.5130 - val_loss: 0.6931 - val_accuracy: 0.5140 - 429ms/epoch - 21ms/step
Ep

KeyboardInterrupt: 